In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy.spatial import distance

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook

from datetime import datetime

In [2]:
mtx = np.array([[9.842439e+02,0,6.900000e+02],
                [0,9.808141e+02,2.331966e+02],
                [0,0,1]])

### Initialize odometry

In [3]:
def load_images(dataset='straight'):
    if dataset == 'straight':
        imgs = glob.glob('./data/straight/*.png')
    elif dataset == 'bicycle':
        imgs = glob.glob('./data/bicycle/*.png')
        oxts = glob.glob('./data/bicycle/oxts/*.txt')
        with open('./data/bicycle/timestamps.txt') as f: time = f.readlines()
    elif dataset == 'highway':
        imgs = glob.glob('./data/highway/*.png')
    elif dataset == 'lidl':
        imgs = glob.glob('./data/lidl/*.png')
    else:
        raise Exception('Invalid dataset')
    assert imgs, 'Could not locate images'
    assert oxts, 'Could not locate oxts measurements'
    #assert time, 'Could not locate timestamps'
    print(f'Images: {len(imgs)}')
    return imgs, oxts, time

In [ ]:
import sys
print(sys.version)

In [5]:
imgs, oxts, time = load_images(dataset='bicycle')

# prepare live plot
output_notebook()


position = dict(x=[0], y=[0])
position2 = dict(x=[0], y=[0])
pos_data = ColumnDataSource(data=dict(x=[0], y=[0]))
pos_data2 = ColumnDataSource(data=dict(x=[0], y=[0]))

pos_figure = figure(plot_width=800, plot_height=400, x_range=(-400, 400), y_range=(-100, 800))
pos_figure.scatter("x", "y", source=pos_data)
pos_figure.scatter("x", "y", source=pos_data2, color='red')
handle = show(pos_figure, notebook_handle=True)

mapping = []


# P_old = np.zeros((3,1))
R_old = np.eye(3)
t_old = np.zeros(3)[:,np.newaxis]
P_old = np.hstack((R_old, t_old))
t_old2 = t_old.copy()
t_save = t_old.copy()
R_save = R_old.copy()

vel_old = t_old.copy()
vel = t_old.copy()

total = len(imgs)

try:
    for idx, fname in enumerate(imgs):
        img_new = cv2.imread(fname, 0)
        if idx == 0: #Reading first frame
            img_old = img_new
            
            time_old = datetime.strptime(time[idx][:-1].split()[1][:-3], "%H:%M:%S.%f")
            with open(oxts[idx]) as f:
                string = f.readlines()[0].split()
                vel_old[2] = float(string[8])
                vel_old[0] = float(string[9])
            continue

        F, pts1, pts2 = matchFeatures(img_new, img_old, feature_ratio=0.7)
        # sensitive to feature_ratio ok values (0.8, 0.7, 0.6) at 0.5 angle of a turn is too small
        P, R, t, relative_scale = updatePos(P_old, F, pts1, pts2, mtx)

        if idx > 0:
            P1 = P # np.hstack((R, t))
            P2 = P_old # np.hstack((R_old, t_old))
            Q = triangulate(pts1.T, P1, pts2.T, P2)
#             Q = cv2.triangulatePoints(P1, P2, pts1.T, pts2.T)  kernel dies
#             reject points behind camera
#             Q = Q[Q[:, 2] > 0]
#             reject points too small
#             Q = Q[np.abs(Q[:, 0]) > 1]
#             Q = Q[np.abs(Q[:, 1]) > 1]
#             Q = Q[np.abs(Q[:, 2]) > 1]
#             reject points too far from camera
#             dist = np.linalg.norm(Q.T - t, axis=0)
#             Q = Q[dist < 200]
            mapping.append(Q)
        
        # vizualize positions|
        position['x'] = t_old[0]
        position['y'] = t_old[2]
        position2['x'] = t_old2[0]
        position2['y'] = t_old2[2]
        pos_data.stream(position, len(imgs))
        pos_data2.stream(position2, len(imgs))
        push_notebook(handle=handle)
        # visualize epilines
        try:
            display_epilines(F, pts1, pts2, img_new, img_old, idx, total, lines_on=True)
        except Exception as e:
            pass
        
        timetemp = datetime.strptime(time[idx][:-1].split()[1][:-3], "%H:%M:%S.%f")
        dt = (timetemp-time_old).total_seconds()
        time_old = timetemp
        with open(oxts[idx]) as f:
            string = f.readlines()[0].split()
            vel[2] = float(string[8])
            vel[0] = float(string[9])
            
        # https://github.com/uoip/monoVO-python/blob/master/visual_odometry.py#L83
        img_old = img_new
        t_save = t
        R_save = R
        t_old = t_old + R_old@(dt*(vel_old + vel)/2)
        t_old2 = t_old2 + R_old@t
        R_old = R@R_old
        P_old = P
        vel_old = vel
except Exception as e:
    cv2.destroyAllWindows()
    raise e

cv2.destroyAllWindows()

Images: 154


Loading BokehJS ...

In [ ]:
cv2.destroyAllWindows()

In [ ]:
# visual odometry

# do we need to rescale x, y to match with real values (in meters) 
#         use camera matrix?
#         velocity from dataset to scale it?

# do we need to estimate camera pose

# mapping
#        triangulate features

# IMU beyond the scope of the course

# Compare to ground truth

# presentation ?

Notes about algorithm

- Sensitive to significant turns
- Sometimes requires reruns
- When vehicle stops at cross-section and other cars are moving the position of camera changes
- Investigate changes in feature_ratio
- Relative scale doesnt change results

### Functions

In [4]:
def drawlines(img1, img2, lines, pts1, pts2, lines_on):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c = img1.shape
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    overlay = img1.copy()
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        if lines_on:
            cv2.line(overlay, (x0,y0), (x1,y1), color, 1)
        color = (0, 255, 100)
        thickness = 1
        radius = 3
        img1 = cv2.circle(img1,tuple(pt1),radius,color,thickness)
        img2 = cv2.circle(img2,tuple(pt2),radius,color,thickness)
    alpha = 0.4
    img1 = cv2.addWeighted(overlay, alpha, img1, 1 - alpha, -1)
    return img1, img2

def display_epilines(F, pts1, pts2, img1, img2, idx, total, lines_on=True):
    lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2, F)
    lines1 = lines1.reshape(-1,3)
    img5,img6 = drawlines(img1, img2, lines1, pts1, pts2, lines_on)
    # Find epilines corresponding to points in left image (first image) and
    # drawing its lines on right image
    lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1, F)
    lines2 = lines2.reshape(-1,3)
    img3,img4 = drawlines(img2, img1, lines2, pts2, pts1, lines_on)
    img = np.vstack((img5,img3))
    # Additional info
    font = cv2.FONT_HERSHEY_DUPLEX
    color = (0, 255, 0)
    coord = (1010, 700)
    cv2.putText(img, f'#Matches: {len(pts1)}', coord, font, 0.5, color, 1, cv2.LINE_AA)
    coord = (1010, 720)
    cv2.putText(img, f'#Frame: {idx}/{total}', coord, font, 0.5, color, 1, cv2.LINE_AA)
    cv2.imshow("Features", img)
    cv2.waitKey(1)

def matchFeatures(img1, img2, feature_ratio=0.3):
    sift = cv2.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    pts1 = []
    pts2 = []
    # ratio test as per Lowe's paper
    for i,(m, n) in enumerate(matches):
        if m.distance < feature_ratio * n.distance:
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)
    # We select only inlier points
    pts1 = pts1[mask.ravel()==1]
    pts2 = pts2[mask.ravel()==1]
    return F, pts1, pts2
    
def updatePos(P, F, pts1, pts2, K):
    E, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.8, threshold=1.0)
    
    num = np.sqrt(pts1[:,0] ** 2 + pts2[:,0] ** 2)
    den = np.sqrt(pts1[:,1] ** 2 + pts2[:,1] ** 2)
    relative_scale = np.median(num / den)
    
    _, R, t, mask = cv2.recoverPose(E, pts1, pts2, K)
#     t = t + relative_scale * R.dot(t) 
#     P = relative_scale * R@P + t
    P = K @ np.hstack((R, t))
    return P, R, t, relative_scale


def triangulate(q1, P1, q2, P2):
    Q = np.zeros((len(q1), 3))
    for i in range(len(q1)):
        B1 = np.vstack((P1[2,:] * q1[i, 0]-P1[0,:],
                        P1[2,:] * q1[i, 1]-P1[1,:]))
        B2 = np.vstack((P2[2,:] * q2[i, 0]-P2[0,:],
                        P2[2,:] * q2[i, 1]-P2[1,:]))
        B = np.vstack((B1, B2))
        U, S, V = np.linalg.svd(B, full_matrices=False)
        Qt = V[-1, :]
        Qt = Qt/Qt[3]
        Q[i, :] = Qt[:3]
    return Q

In [ ]:
new_mapping.shape

In [ ]:
new_mapping = []
for frame in mapping:
    for point in frame:
        new_mapping.append(point)
new_mapping = np.array(new_mapping)

In [ ]:
def median(x):
    m,n = x.shape
    middle = np.arange((m-1)>>1,(m>>1)+1)
    x = np.partition(x,middle,axis=0)
    return x[middle].mean(axis=0)

# main function
def remove_outliers(data,thresh=2.0):           
    m = median(data)                            
    s = np.abs(data-m)                          
    return data[(s<median(s)*thresh).all(axis=1)]

In [ ]:
new_mapping.shape

In [ ]:
remove_outliers(new_mapping).shape

In [ ]:
import open3d as o3d

depn = o3d.geometry.PointCloud()
depn.points = o3d.utility.Vector3dVector(remove_outliers(new_mapping))
o3d.visualization.draw_geometries_with_editing([depn])